In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np

In [24]:
df = pd.read_csv('./data/clean_data_clustered.csv')
df

,Unnamed: 0,index,county,state,season,title,latitude,longitude,date,number,...,humidity,cloud_cover,precip_type,pressure,wind_bearing,wind_speed,Class A,Class B,Class C,cluster
0,0,2,Washington County,Rhode Island,Fall,Report 6496: Bicycling student has night encou...,41.45000,-71.50000,1974-09-20,6496.0,...,0.907111,0.268375,rain,1019.476257,263.869568,11.258579,1,0,0,6
1,1,5,Washita County,Oklahoma,Fall,Report 9765: Motorist and children have daylig...,35.30110,-99.17020,1973-09-28,9765.0,...,0.646660,0.113750,No Precipitation,953.307434,164.583038,14.922651,1,0,0,2
2,2,6,Washington County,Ohio,Summer,Report 4983: Sighting and vocalizations at a h...,39.38745,-81.67339,1971-08-01,4983.0,...,0.745234,0.584542,No Precipitation,993.191711,17.920425,4.966946,1,0,0,1
3,3,7,Westchester County,New York,Fall,Report 31940: Fishermen and Law Enforcement Of...,41.29873,-73.73717,2010-09-01,31940.0,...,0.578789,0.126625,No Precipitation,1001.331787,270.616058,6.397976,1,0,0,6
4,4,8,Washoe County,Nevada,Fall,Report 5692: Father and son hear vocalizations...,39.59200,-119.97500,1970-09-01,5692.0,...,0.402105,0.000000,No Precipitation,807.823975,219.053116,9.812248,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034,4034,5010,Harlan County,Kentucky,Fall,Report 25257: ATV riders find possible tracks ...,36.89771,-83.16293,2008-10-15,25257.0,...,0.740113,0.151125,No Precipitation,911.663757,201.991241,5.410581,0,1,0,1
4035,4035,5011,Harlan County,Kentucky,Winter,Report 38980: Teen on four-wheeler has sightin...,36.83193,-83.42429,2013-01-10,38980.0,...,0.930158,0.878042,rain,980.778503,89.679916,6.219504,1,0,0,1
4036,4036,5012,Anderson County,Kentucky,Fall,Report 1045: Father and son have a daylight en...,37.95708,-84.97527,2005-01-09,1045.0,...,0.793257,0.548708,No Precipitation,995.131042,178.840332,7.880466,1,0,0,1
4037,4037,5013,Anderson County,Kentucky,Fall,Report 33187: Dual sighting in late afternoon ...,37.97699,-84.90234,2011-11-15,33187.0,...,0.936365,0.963083,rain,984.935791,201.164108,13.016956,1,0,0,1


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4039 entries, 0 to 4038
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        4039 non-null   int64  
 1   index             4039 non-null   int64  
 2   county            4039 non-null   object 
 3   state             4039 non-null   object 
 4   season            4039 non-null   object 
 5   title             4039 non-null   object 
 6   latitude          4039 non-null   float64
 7   longitude         4039 non-null   float64
 8   date              4039 non-null   object 
 9   number            4039 non-null   float64
 10  geohash           4039 non-null   object 
 11  temperature_high  4039 non-null   float64
 12  temperature_mid   4039 non-null   float64
 13  temperature_low   4039 non-null   float64
 14  dew_point         4039 non-null   float64
 15  humidity          4039 non-null   float64
 16  cloud_cover       4039 non-null   float64


In [26]:
season_d = pd.get_dummies(df['season'], dtype=int)
rain_d = pd.get_dummies(df['precip_type'], dtype=int)
df.drop(['season', 'index', 'county', 'state', 'title', 'latitude', 'longitude', 'date', 'geohash', 'precip_type'], axis=1, inplace=True)
df = pd.concat([df, season_d, rain_d], axis=1)
df

,Unnamed: 0,number,temperature_high,temperature_mid,temperature_low,dew_point,humidity,cloud_cover,pressure,wind_bearing,...,Class C,cluster,Fall,Spring,Summer,Unknown,Winter,No Precipitation,rain,snow
0,0,6496.0,74.165901,69.148399,66.515900,66.227150,0.907111,0.268375,1019.476257,263.869568,...,0,6,1,0,0,0,0,0,1,0
1,1,9765.0,70.798103,59.698109,52.708099,47.139355,0.646660,0.113750,953.307434,164.583038,...,0,2,1,0,0,0,0,1,0,0
2,2,4983.0,77.978302,69.630798,61.328300,60.795803,0.745234,0.584542,993.191711,17.920425,...,0,1,0,0,1,0,0,1,0,0
3,3,31940.0,93.406998,81.999496,70.097000,63.793255,0.578789,0.126625,1001.331787,270.616058,...,0,6,1,0,0,0,0,1,0,0
4,4,5692.0,76.099098,59.674103,45.499100,29.029099,0.402105,0.000000,807.823975,219.053116,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034,4034,25257.0,73.876099,63.743607,53.626099,54.413593,0.740113,0.151125,911.663757,201.991241,...,0,1,1,0,0,0,0,1,0,0
4035,4035,38980.0,61.499302,49.656803,43.859299,47.541798,0.930158,0.878042,980.778503,89.679916,...,0,1,0,0,0,0,1,0,1,0
4036,4036,1045.0,52.062798,40.152798,33.882801,33.796551,0.793257,0.548708,995.131042,178.840332,...,0,1,1,0,0,0,0,1,0,0
4037,4037,33187.0,68.785698,58.773205,54.295700,56.680698,0.936365,0.963083,984.935791,201.164108,...,0,1,1,0,0,0,0,0,1,0


In [27]:
labels = df.filter(['cluster'], axis=1)
features = df.drop(['cluster', 'Class A', 'Class B', 'Class C'], axis=1)

In [28]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.3, random_state=42)

In [29]:
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')
clf.fit(train_x, train_y)
score = clf.score(test_x, test_y)
print("Accuracy:", score)

Accuracy: 0.3976897689768977


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
ss = StandardScaler()
train_x = ss.fit_transform(train_x)
test_x = ss.transform(test_x)

In [31]:
reg = LinearRegression()
reg.fit(train_x, train_y)

# create parameters needed for lasso and ridge
parameters = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'tol': [0.0001, 0.001, 0.01, 0.1], 'max_iter': [500, 1000, 2000, 4000, 8000]}

las = Lasso() # create our lasso model
lasso = GridSearchCV(las, parameters, scoring='neg_mean_squared_error', cv=10)
lasso.fit(train_x, train_y)

rid = Ridge() # create our ridge model
ridge = GridSearchCV(rid, parameters, scoring='neg_mean_squared_error', cv=10)
ridge.fit(train_x, train_y)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.491e+01, tolerance: 1.164e+00
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.491e+01, tolerance: 1.161e+00
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.652e+00, toler

GridSearchCV(cv=10, estimator=Ridge(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'max_iter': [500, 1000, 2000, 4000, 8000],
                         'tol': [0.0001, 0.001, 0.01, 0.1]},
             scoring='neg_mean_squared_error')

In [32]:
print("Linear Regression: " + str(np.sqrt(mean_squared_error(test_y, reg.predict(test_x)))))
print("LASSO Regression: " + str(np.sqrt(mean_squared_error(test_y, lasso.predict(test_x)))))
print("Ridge Regression: " + str(np.sqrt(mean_squared_error(test_y, ridge.predict(test_x)))))

Linear Regression: 2.077177888036041
LASSO Regression: 2.0792811843806027
Ridge Regression: 2.0793915397405027


In [33]:
labels = df.filter(['Class A'], axis=1)
features = df.drop(['Class A', 'Class B', 'Class C'], axis=1)

In [34]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.3, random_state=42)

In [35]:
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')
clf.fit(train_x, train_y)
score = clf.score(test_x, test_y)
print("Accuracy:", score)

Accuracy: 0.5841584158415841


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
ss = StandardScaler()
train_x = ss.fit_transform(train_x)
test_x = ss.transform(test_x)

In [37]:
reg = LinearRegression()
reg.fit(train_x, train_y)

# create parameters needed for lasso and ridge
parameters = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'tol': [0.0001, 0.001, 0.01, 0.1], 'max_iter': [500, 1000, 2000, 4000, 8000]}

las = Lasso() # create our lasso model
lasso = GridSearchCV(las, parameters, scoring='neg_mean_squared_error', cv=10)
lasso.fit(train_x, train_y)

rid = Ridge() # create our ridge model
ridge = GridSearchCV(rid, parameters, scoring='neg_mean_squared_error', cv=10)
ridge.fit(train_x, train_y)

GridSearchCV(cv=10, estimator=Ridge(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'max_iter': [500, 1000, 2000, 4000, 8000],
                         'tol': [0.0001, 0.001, 0.01, 0.1]},
             scoring='neg_mean_squared_error')

In [38]:
print("Linear Regression: " + str(np.sqrt(mean_squared_error(test_y, reg.predict(test_x)))))
print("LASSO Regression: " + str(np.sqrt(mean_squared_error(test_y, lasso.predict(test_x)))))
print("Ridge Regression: " + str(np.sqrt(mean_squared_error(test_y, ridge.predict(test_x)))))

Linear Regression: 0.49183868482533566
LASSO Regression: 0.49180252183485884
Ridge Regression: 0.49225036824433444
